In [5]:
import pandas as pd 
import os

In [6]:
folder_path= './PassengerVehicle_Stats'
dfs=[]
files= [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for file in files:
    file_path = os.path.join(folder_path,file)
    df1 = pd.read_csv(file_path)
    dfs.append(df1)

combine_df = pd.concat(dfs, ignore_index=True)
combine_df.to_csv('./vehicles_df.csv', index= False)

In [7]:
df = pd.read_csv('vehicles_df.csv')
df.isna().sum()

Unnamed: 0                                0
Public Vehicle Number                     0
Status                                    0
Vehicle Make                           7668
Vehicle Model                          7852
Vehicle Model Year                     7768
Vehicle Color                          7944
Vehicle Fuel Source                       0
Wheelchair Accessible                     0
Company Name                              0
Address                                7144
City                                   7144
State                                  7144
ZIP Code                               7144
Taxi Affiliation                      37016
Taxi Medallion License Management     37124
Record ID                                 0
dtype: int64

In [10]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66408 entries, 0 to 66407
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          66408 non-null  int64  
 1   Public Vehicle Number               66408 non-null  int64  
 2   Status                              66408 non-null  object 
 3   Vehicle Make                        58740 non-null  object 
 4   Vehicle Model                       58556 non-null  object 
 5   Vehicle Model Year                  58640 non-null  float64
 6   Vehicle Color                       58464 non-null  object 
 7   Vehicle Fuel Source                 66408 non-null  object 
 8   Wheelchair Accessible               66408 non-null  object 
 9   Company Name                        66408 non-null  object 
 10  Address                             59264 non-null  object 
 11  City                                59264